<a href="https://colab.research.google.com/github/Soumi7/TFNotebooks/blob/master/UCF101_specific_folder_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import random
import os
import ssl
import cv2
import numpy as np
import imageio
from IPython import display
from urllib import request
import re
import tempfile

In [3]:
UCF_ROOT = "https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/"
_VIDEO_LIST = None
Videos ={}
_CACHE_DIR = tempfile.mkdtemp()
# default Colab environment anymore.
unverified_context = ssl._create_unverified_context()
def list_ucf_videos():
  global _VIDEO_LIST
  if not _VIDEO_LIST:
    index = request.urlopen(UCF_ROOT, context=unverified_context).read().decode("utf-8")
    v = ['HandstandPushups','HandstandWalking','PullUps','Punch','PushUps']
    for i in v:
      s = "(v_"+str(i)+"[\w_]+\.avi)"
      videos = re.findall(s, index)
      Videos[i]=sorted(set(videos))
  return Videos

In [4]:
V={}
V = list_ucf_videos()
for key,value in V.items():
  print(key,value)

HandstandPushups ['v_HandstandPushups_g01_c01.avi', 'v_HandstandPushups_g01_c02.avi', 'v_HandstandPushups_g01_c03.avi', 'v_HandstandPushups_g01_c04.avi', 'v_HandstandPushups_g02_c01.avi', 'v_HandstandPushups_g02_c02.avi', 'v_HandstandPushups_g02_c03.avi', 'v_HandstandPushups_g02_c04.avi', 'v_HandstandPushups_g03_c01.avi', 'v_HandstandPushups_g03_c02.avi', 'v_HandstandPushups_g03_c03.avi', 'v_HandstandPushups_g03_c04.avi', 'v_HandstandPushups_g04_c01.avi', 'v_HandstandPushups_g04_c02.avi', 'v_HandstandPushups_g04_c03.avi', 'v_HandstandPushups_g04_c04.avi', 'v_HandstandPushups_g05_c01.avi', 'v_HandstandPushups_g05_c02.avi', 'v_HandstandPushups_g05_c03.avi', 'v_HandstandPushups_g05_c04.avi', 'v_HandstandPushups_g06_c01.avi', 'v_HandstandPushups_g06_c02.avi', 'v_HandstandPushups_g06_c03.avi', 'v_HandstandPushups_g06_c04.avi', 'v_HandstandPushups_g07_c01.avi', 'v_HandstandPushups_g07_c02.avi', 'v_HandstandPushups_g07_c03.avi', 'v_HandstandPushups_g07_c04.avi', 'v_HandstandPushups_g08_c01.av

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# fetch videos from UCF101 dataset
UCF_ROOT = "https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/"
_VIDEO_LIST = V['HandstandPushups']
_CACHE_DIR = "/content/drive/My Drive/ucf/HandstandPushups"

# default Colab environment anymore.
unverified_context = ssl._create_unverified_context()
def fetch_ucf_video(video):
  cache_path = os.path.join(_CACHE_DIR, video)
  if not os.path.exists(cache_path):
    urlpath = request.urljoin(UCF_ROOT, video)
    print("Fetching %s => %s" % (urlpath, cache_path))
    data = request.urlopen(urlpath, context=unverified_context).read()
    open(cache_path, "wb").write(data)
  return cache_path
def crop_center_square(frame):
  y, x = frame.shape[0:2]
  min_dim = min(y, x)
  start_x = (x // 2) - (min_dim // 2)
  start_y = (y // 2) - (min_dim // 2)
  return frame[start_y:start_y+min_dim,start_x:start_x+min_dim]

In [7]:
for i in _VIDEO_LIST:
  video_path = fetch_ucf_video(i)

Fetching https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_HandstandPushups_g01_c01.avi => /content/drive/My Drive/ucf/HandstandPushups/v_HandstandPushups_g01_c01.avi
Fetching https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_HandstandPushups_g01_c02.avi => /content/drive/My Drive/ucf/HandstandPushups/v_HandstandPushups_g01_c02.avi
Fetching https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_HandstandPushups_g01_c03.avi => /content/drive/My Drive/ucf/HandstandPushups/v_HandstandPushups_g01_c03.avi
Fetching https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_HandstandPushups_g01_c04.avi => /content/drive/My Drive/ucf/HandstandPushups/v_HandstandPushups_g01_c04.avi
Fetching https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_HandstandPushups_g02_c01.avi => /content/drive/My Drive/ucf/HandstandPushups/v_HandstandPushups_g02_c01.avi
Fetching https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_HandstandPushups_g02_c02.avi => /content/drive/My Drive/ucf/HandstandPushups/v_HandstandPushups_g02_c02.av